In [28]:
import requests
from bs4 import BeautifulSoup
import json
import re
import urllib
import csv

from selenium import webdriver
import pandas as pd
from selenium.webdriver.common.by import By

In [38]:
url = "https://twitter.com/search?q=github%20arxiv.org&src=typd"

from selenium import webdriver
 
from bs4 import BeautifulSoup

import time
browser = webdriver.Chrome('/Users/shreyasawkar/Downloads/chromedriver')

browser.get(url)
json_data = []

SCROLL_PAUSE_TIME = 1.5

# Get scroll height
last_height = browser.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = browser.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height
    
page = BeautifulSoup(browser.page_source,"html5lib")
browser.close()
links = page.findAll('li',attrs={'class': 'js-stream-item stream-item stream-item'})



In [39]:
len(links)

163

In [40]:
githubUrls = []
arxivUrl = []

line_count = 0;
for li in links:
#     if(line_count>5):
#         break;
    line_count+=1
    githubUrl = ''
    arxivUrl = ''
    allAnchorTags = li.find('div',attrs={'class':'js-tweet-text-container'}).findAll('a')
    for anchor in allAnchorTags:
        try:
            responseAnchor = requests.get(anchor['href'])
            if(str(responseAnchor.url).startswith('https://github.com/')):
                githubUrl = responseAnchor.url
               
            if(str(responseAnchor.url).startswith('https://arxiv.org/')):
                arxivUrl = str(responseAnchor.url).replace('pdf','abs')
#                 arxivUrl.replace('pdf','abs')
        except:
            continue
    json_data.append([githubUrl,arxivUrl])

with open('/Users/shreyasawkar/PycharmProjects/CSCI-548-Project/venv/Task1-DataExtraction/TwitterExtractedUrls.csv', 'w') as csvFile:
    writer = csv.writer(csvFile)
    writer.writerows(json_data)
               


In [58]:
json_data


[['', 'https://arxiv.org/abs/1904.07842'],
 ['https://github.com/AaltoVision/DGC-Net',
  'https://arxiv.org/abs/1810.08393'],
 ['https://github.com/deepmind/lab/tree/master/game_scripts/levels/contributed/dmlab30',
  'https://arxiv.org/abs/1802.01561'],
 ['https://github.com/tttthomasssss/iwcs2019',
  'https://arxiv.org/abs/1904.01297.abs'],
 ['https://github.com/AtsushiSakai/PythonRobotics',
  'https://arxiv.org/abs/1808.10703'],
 ['https://github.com/deepmind/hanabi-learning-environment',
  'https://arxiv.org/abs/1902.00506'],
 ['', 'https://arxiv.org/abs/1808.01560.abs'],
 ['https://github.com/mandubian/neural-ode',
  'https://arxiv.org/abs/1806.07366'],
 ['', 'https://arxiv.org/abs/1811.02021'],
 ['https://github.com/sangwoomo/instagan', 'https://arxiv.org/abs/1812.10889'],
 ['https://github.com/facebookresearch/UnsupervisedMT/blob/master/README.md',
  'https://arxiv.org/abs/1804.07755'],
 ['https://github.com/keras-team/autokeras',
  'https://arxiv.org/abs/1806.10282'],
 ['https:/

In [67]:
finalTwitterData = []
line_count = 0
Urls = []
for row in json_data:
    try:
#         line_count+=1
#         if(line_count>10):
#             break;
            
        if(row[0]!=''):
#             githubUrl = row[0]
#             print(row[0])
#             print("-------------------")

            githubUrl = row[0]+"/blob/master/README.md"
        #             print(githubUrl)
            response = requests.get(githubUrl)
            html = response.content
            soup = BeautifulSoup(html, "html.parser")
            rawUrl = soup.find('a',attrs={'id': 'raw-url'})['href']
            print(rawUrl)
            response = requests.get('https://github.com'+rawUrl)
            html = response.content
            soup = BeautifulSoup(html, "html.parser")
            githubReadme = soup
            githubReadme = str(soup).rstrip("\n\r").replace(",","")
#             print(soup)
        

        
        arxivUrl= row[1]
        if(str(arxivUrl)=='' or str(arxivUrl)=='https://arxiv.org/'):
            continue;
        response = requests.get(arxivUrl)
        html = response.content
        soup = BeautifulSoup(html, "html.parser")
        abstract = soup.find('blockquote',attrs={'class':'abstract mathjax'}).text
#         print(abstract)
        classifierText =  str(abstract) + githubReadme
        classifierText = classifierText.replace(",","")
        Urls.append([githubUrl,arxivUrl,abstract])
        finalTwitterData.append([classifierText])
        
    except Exception as e:
#         print(e)
        continue 

               

/AaltoVision/DGC-Net/raw/master/README.md
/AtsushiSakai/PythonRobotics/raw/master/README.md
/deepmind/hanabi-learning-environment/raw/master/README.md
/mandubian/neural-ode/raw/master/README.md
/sangwoomo/instagan/raw/master/README.md
/keras-team/autokeras/raw/master/README.md
/facebookresearch/agem/raw/master/README.md
/andrew-j-levy/Hierarchical-Actor-Critc-HAC-/raw/master/README.md
/wuhuikai/FastFCN/raw/master/README.md
/julienrf/endpoints/raw/master/README.md
/SoheilFeizi/MCPCA/raw/master/README.md
/deepmind/hanabi-learning-environment/raw/master/README.md
/rois-codh/kmnist/raw/master/README.md
/facebookresearch/pythia/raw/master/README.md
/bloomberg/cnn-rnf/raw/master/README.md
/aqlaboratory/proteinnet/raw/master/README.md
/evaluating-adversarial-robustness/adv-eval-paper/raw/master/README.md
/rahulbhalley/cyclegan-qp/raw/master/README.md
/fangohr/paper-supplement-standard-problem-dmi/raw/master/README.md
/IAIK/sgxrop/raw/master/README.md
/uber-research/go-explore/raw/master/READM

In [68]:
len(finalTwitterData)

111

In [69]:


with open('/Users/shreyasawkar/PycharmProjects/CSCI-548-Project/venv/Task1-DataExtraction/FinalTwitterData1.csv', 'w') as csvFile:
    writer = csv.writer(csvFile)
    writer.writerows(finalTwitterData)

    

with open('/Users/shreyasawkar/PycharmProjects/CSCI-548-Project/venv/Task1-DataExtraction/UrlsData.csv', 'w') as csvFile:
    writer = csv.writer(csvFile)
    writer.writerows(Urls)
    
    